In [4]:
# Soil Classification - Final Corrected Notebook

## 1. Imports and Setup
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.serialization import safe_globals
import numpy.core.multiarray
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 2. Dataset Classes
class SoilDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = img_dir
        self.transform = transform
        self.classes = sorted(self.data['soil_type'].unique())
        self.class_to_idx = {label: idx for idx, label in enumerate(self.classes)}
        self.idx_to_class = {idx: label for label, idx in self.class_to_idx.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.image_dir, row['image_id'])
        image = np.array(Image.open(img_path).convert("RGB"))
        label = self.class_to_idx[row['soil_type']]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label

class TestDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['image_id']
        img_path = os.path.join(self.image_dir, img_name)
        image = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            image = self.transform(image=image)['image']
        return image, img_name

## 3. Corrected Transforms
train_transform = A.Compose([
    A.SmallestMaxSize(max_size=256),
    A.RandomCrop(height=224, width=224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.SmallestMaxSize(max_size=256),
    A.CenterCrop(height=224, width=224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

## 4. Data Loading
train_dir = '/kaggle/input/soilclassification1/soil_classification-2025/train'
test_dir = '/kaggle/input/soilclassification1/soil_classification-2025/test'
train_csv = '/kaggle/input/soilclassification1/soil_classification-2025/train_labels.csv'
test_csv = '/kaggle/input/soilclassification1/soil_classification-2025/test_ids.csv'

full_dataset = SoilDataset(train_csv, train_dir, transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Apply val transform to validation set
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

## 5. Model Setup
model = timm.create_model('tf_efficientnet_b4', pretrained=True, num_classes=len(full_dataset.classes))
model = model.to(device)

# Loss function with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Optimizer with weight decay
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

# Cosine annealing learning rate scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-6)

## 6. Training Loop
num_epochs = 25
best_f1 = 0

for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_acc = 100 * correct / total
    
    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for val_images, val_labels_batch in val_loader:
            val_images = val_images.to(device)
            val_labels_batch = val_labels_batch.to(device)
            val_outputs = model(val_images)
            _, val_pred = val_outputs.max(1)
            val_preds.extend(val_pred.cpu().numpy())
            val_labels.extend(val_labels_batch.cpu().numpy())
    
    # Calculate F1 score
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    scheduler.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Acc: {train_acc:.2f}% | Val F1: {val_f1:.4f}")
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'f1': val_f1,
            'epoch': epoch
        }, 'best_soil_model.pth')
        print(f"Model saved with F1: {val_f1:.4f}")



/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_35/343854825.py:71: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.5),


Epoch [1/25] | Train Acc: 78.20% | Val F1: 0.8700
Model saved with F1: 0.8700
Epoch [2/25] | Train Acc: 91.91% | Val F1: 0.9346
Model saved with F1: 0.9346
Epoch [3/25] | Train Acc: 96.42% | Val F1: 0.9593
Model saved with F1: 0.9593
Epoch [4/25] | Train Acc: 98.67% | Val F1: 0.9472
Epoch [5/25] | Train Acc: 99.39% | Val F1: 0.9674
Model saved with F1: 0.9674
Epoch [6/25] | Train Acc: 99.59% | Val F1: 0.9594
Epoch [7/25] | Train Acc: 99.59% | Val F1: 0.9675
Model saved with F1: 0.9675
Epoch [8/25] | Train Acc: 99.80% | Val F1: 0.9675
Epoch [9/25] | Train Acc: 99.80% | Val F1: 0.9594
Epoch [10/25] | Train Acc: 99.39% | Val F1: 0.9715
Model saved with F1: 0.9715
Epoch [11/25] | Train Acc: 99.80% | Val F1: 0.9552
Epoch [12/25] | Train Acc: 99.59% | Val F1: 0.9796
Model saved with F1: 0.9796
Epoch [13/25] | Train Acc: 99.69% | Val F1: 0.9716
Epoch [14/25] | Train Acc: 99.80% | Val F1: 0.9633
Epoch [15/25] | Train Acc: 99.80% | Val F1: 0.9796
Epoch [16/25] | Train Acc: 99.90% | Val F1: 0.97

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms


In [ ]:
# Example paths (adjust if needed based on unzip location)
data_dir = "/kaggle/input/soil-classification-part-2/soil_competition-2025"
train_csv = os.path.join(data_dir, "train_labels.csv")
train_img_dir = os.path.join(data_dir, "train")
test_img_dir = os.path.join(data_dir, "test")


In [ ]:
df = pd.read_csv(train_csv)

In [ ]:
train_df, val_df = train_test_split(df_balanced, test_size=0.2, stratify=df_balanced['label'], random_state=42)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
batch_size = 32
train_loader = DataLoader(SoilDataset(train_df, train_img_dir, transform=transform_train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(SoilDataset(val_df, train_img_dir, transform=transform_test), batch_size=batch_size)
test_loader = DataLoader(SoilDataset(pd.DataFrame({'image_id': os.listdir(test_img_dir)}), test_img_dir, transform=transform_test, test=True), batch_size=batch_size)

In [ ]:
def train_model(model, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {total_loss/len(train_loader):.4f}")
        evaluate_model(model)
        scheduler.step()

In [ ]:
train_model(model, epochs=10)